## 1. Prepare data

In [1]:
import json
from collections import OrderedDict
from pprint import pprint

with open('/home/taeho/data/coco2014/annotations/annotations/instances_val2014.json', encoding="utf-8") as data_file:    
    data = json.load(data_file, object_pairs_hook=OrderedDict)

#pprint(data) #data는 json 전체를 dictionary 형태로 저장하고 있음

for i in range(0,50):
    image_id = data["images"][i]["id"]
    image_height = data["images"][i]["height"]
    image_width = data["images"][i]["width"]
    print("image_id =", image_id)
    
    for index, name in enumerate(data["annotations"]):
        category_id = name["category_id"]
        bbox = name["bbox"]
        
        if name["image_id"] == image_id:
            #print(name["image_id"])
            
            for index, name in enumerate(data["categories"]):
                
                if name["id"] == category_id:
                    #print(name["id"])
                    #print()
                    ##print(category_id, name["name"])
                    ##print(bbox)
                    pass
            
    #print()

#for i in range(0,100):
#    print(data["categories"][i]["id"])
#    print(data["categories"][i]["name"])


image_id = 391895
image_id = 522418
image_id = 184613
image_id = 318219
image_id = 554625
image_id = 397133
image_id = 574769
image_id = 60623
image_id = 309022
image_id = 5802
image_id = 222564
image_id = 118113
image_id = 193271
image_id = 224736
image_id = 483108
image_id = 403013
image_id = 374628
image_id = 328757
image_id = 384213
image_id = 293802
image_id = 86408
image_id = 37777
image_id = 372938
image_id = 386164
image_id = 223648
image_id = 204805
image_id = 113588
image_id = 384553
image_id = 337264
image_id = 368402
image_id = 12448
image_id = 252219
image_id = 79841
image_id = 87038
image_id = 174482
image_id = 515289
image_id = 562150
image_id = 542145
image_id = 412151
image_id = 403385
image_id = 579003
image_id = 540186
image_id = 242611
image_id = 51191
image_id = 269105
image_id = 294832
image_id = 462565
image_id = 144941
image_id = 173350
image_id = 60760


## 2. Train

In [1]:
import os
import itertools
import torch





from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

from utils.misc import str2bool, Timer, freeze_net_layers, store_labels

from ssd.ssd import MatchPrior
from ssd.vgg_ssd import create_vgg_ssd
from ssd.config import vgg_ssd_config
from ssd.data_preprocessing import TrainAugmentation, TestTransform

from datasets.COCO_dataset import COCODataset
#from datasets.NEU_dataset import VOCDataset

from nn.multibox_loss import MultiboxLoss


print('Single Shot MultiBox Detector Training With Pytorch')

dataset_type = "COCO"
datasets = "/home/taeho/data/coco2014"
validation_dataset = "/home/taeho/data/coco2014"
net = "vgg16-ssd"

# Params for SGD
lr = 1e-3 #1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
base_net_lr = None
extra_layers_lr = None

# Params for loading pretrained basenet or checkpoints.
base_net = "models/vgg16_reducedfc.pth"
pretrained_ssd = None
resume = None

# Scheduler
scheduler = "multi-step"

# Params for Multi-step Scheduler
milestones = "120,160"

# Params for Cosine Annealing
t_max = 120.0

# Train params
batch_size = 16 #24
num_epochs = 2
num_workers = 4
validation_epochs = 5
debug_steps = 100
use_cuda = True
checkpoint_folder = 'models/'

#
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() and use_cuda else "cpu")

if use_cuda and torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print("Use Cuda.")


def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        
        regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)  # TODO CHANGE BOXES
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        
        if i and i % debug_steps == 0:
            
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0


def test(loader, net, criterion, device):
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num


#timer = Timer()
    
if net == 'vgg16-ssd':
    create_net = create_vgg_ssd
    config = vgg_ssd_config
else:
    print("No network")
    
    
train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance,
                              config.size_variance, 0.5)

test_transform = TestTransform(config.image_size, config.image_mean, config.image_std)


print("Prepare training and validation datasets.")
datalist = []

if dataset_type == 'COCO':

    dataset = COCODataset(datasets, transform=train_transform,
                         target_transform=target_transform, is_test=False)
    val_dataset = COCODataset(validation_dataset, transform=test_transform,
                             target_transform=target_transform, is_test=True)
    
    label_file = os.path.join(checkpoint_folder, "coco-model-labels.txt")
    store_labels(label_file, dataset.class_names)
    num_classes = len(dataset.class_names)
    
    print("Stored labels into file coco-model-labels.txt.")
else:
    print("No dataset_type")

datalist.append(dataset)
train_dataset = ConcatDataset(datalist)

print("Train dataset size: {}". format(len(train_dataset)))
print("validation dataset size: {}".format(len(val_dataset)))

train_loader = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)
val_loader = DataLoader(val_dataset, batch_size,
                        num_workers=num_workers,
                        shuffle=False)



print("Build network.")
net = create_net(num_classes)
min_loss = -10000.0
last_epoch = -1

base_net_lr = base_net_lr if base_net_lr is not None else lr
extra_layers_lr = extra_layers_lr if extra_layers_lr is not None else lr
    
params = [
    {'params': net.base_net.parameters(), 'lr': base_net_lr},
    {'params': itertools.chain(
        net.source_layer_add_ons.parameters(),
        net.extras.parameters()
    ), 'lr': extra_layers_lr},
    {'params': itertools.chain(
        net.regression_headers.parameters(),
        net.classification_headers.parameters()
    )}
]

#timer.start("Load Model")
if resume:
    print("Resume from the model")
    net.load(resume)
elif base_net:
    print("Init from base net")
    net.init_from_base_net(base_net)
elif pretrained_ssd:
    print("Init from pretrained ssd")
    net.init_from_pretrained_ssd(pretrained_ssd)
print()

net.to(DEVICE)

criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                         center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum,
                            weight_decay=weight_decay)
print("Learning rate:", lr, "Base net learning rate:", base_net_lr,
      "Extra Layers learning rate:", extra_layers_lr)

if scheduler == 'multi-step':
    print("Uses MultiStepLR scheduler.")
    milestones = [int(v.strip()) for v in milestones.split(",")]
    scheduler = MultiStepLR(optimizer, milestones=milestones,
                            gamma=0.1, last_epoch=last_epoch)
elif scheduler == 'cosine':
    print("Uses CosineAnnealingLR scheduler.")
    scheduler = CosineAnnealingLR(optimizer, t_max, last_epoch=last_epoch)
else:
    print("Unsupported Scheduler")
    

print("Start training from epoch 0.\n")
for epoch in range(last_epoch + 1, num_epochs):
    scheduler.step()
    train(train_loader, net, criterion, optimizer,
          device=DEVICE, debug_steps=debug_steps, epoch=epoch)
        
    if epoch % validation_epochs == 0 or epoch == num_epochs - 1:
        val_loss, val_regression_loss, val_classification_loss = test(val_loader, net, criterion, DEVICE)
        print("Epoch:", epoch, "\n", 
              "Validation Loss:", val_loss, "\n",
              "Validation Regression Loss:", val_regression_loss, "\n",
              "Validation Classification Loss:", val_classification_loss
              )
        
        name = "vgg16-ssd" + "-Epoch-" + "%s"%epoch + "-Loss-" + "%s"%str(val_loss)[:7] + ".pth"
        model_path = os.path.join(checkpoint_folder, name)
        ##model_path = os.path.join(checkpoint_folder, f"{net}-Epoch-{epoch}-Loss-{val_loss}.pth")
        net.save(model_path)
        print("Saved model:", model_path, "\n")
        


Single Shot MultiBox Detector Training With Pytorch
Use Cuda.
Prepare training and validation datasets.
Stored labels into file coco-model-labels.txt.
Train dataset size: 24
validation dataset size: 24
Build network.
Init from base net

Learning rate: 0.001 Base net learning rate: 0.001 Extra Layers learning rate: 0.001
Uses MultiStepLR scheduler.
Start training from epoch 0.

Epoch: 0 
 Validation Loss: 43.81428337097168 
 Validation Regression Loss: 5.715577602386475 
 Validation Classification Loss: 38.09870719909668
Saved model: models/vgg16-ssd-Epoch-0-Loss-43.8142.pth 

Epoch: 1 
 Validation Loss: 28.753564834594727 
 Validation Regression Loss: 4.5066680908203125 
 Validation Classification Loss: 24.246896743774414
Saved model: models/vgg16-ssd-Epoch-1-Loss-28.7535.pth 

